In [ ]:
import numpy as np
import pandas as pd
import tejapi

tejapi.ApiConfig.api_key  = "Your Key"
tejapi.ApiConfig.api_base = "https://api.tej.com.tw"
tejapi.ApiConfig.ignoretz = True

In [3]:
file_path ='/財務資訊重點專區.csv'
sheet_name = '工作表1'
df = pd.read_csv(file_path)
df['年月日'] = pd.to_datetime(df['年月日'], format='%Y%m%d')
start_date = '2022-01-03'
end_date = '2024-06-07'
mask_date = (df['年月日'] >= start_date) & (df['年月日'] <= end_date)
mask_market = df['市場別'] == '上市'
filtered_df = df[mask_date & mask_market] #僅挑選上市公司
filtered_df

C:\Users\to097\AppData\Local\Temp\ipykernel_31672\4229850982.py:3: DtypeWarning: Columns (9,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,證券代碼,簡稱,年月日,市場別,每股淨值,符合指標1 (Y/N),指標1_交易方式,符合指標2 (Y/N),指標2_最近稅後損益年月1,指標2_最近稅後損益年月2,...,指標4_最近營業活動年月3,指標4_最近期營業活動現金流量1,指標4_最近期營業活動現金流量2,指標4_最近期營業活動現金流量3,符合指標5 (Y/N),指標5_資金貸與佔淨值比,符合指標6 (Y/N),指標6_背書保證佔淨值比,符合指標7 (Y/N),指標7_其他
0,1213,大飲,2022-01-03,上市,8.10,Y,變更交易,Y,107,108,...,110Q3,-67444.0,-3781.0,-36780,Y,30.73,N,0.00,Y,有
1,1235,興泰,2022-01-03,上市,33.15,N,普通交易,N,107,108,...,110Q3,41647.0,134987.0,63082,N,0.00,N,0.00,Y,有
3,1337,再生-KY,2022-01-03,上市,19.90,N,普通交易,N,107,108,...,110Q3,-340717.0,-215473.0,-266696,N,0.00,N,0.00,Y,有
4,1340,勝悅-KY,2022-01-03,上市,33.54,N,普通交易,N,107,108,...,110Q3,-194433.0,-312789.0,-232844,N,0.00,N,0.00,Y,有
5,1413,宏洲,2022-01-03,上市,8.34,N,普通交易,N,107,108,...,110Q3,5974.0,-39246.0,111106,N,0.00,N,0.00,N,無
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184252,8404,百和興業-KY,2024-06-07,上市,15.48,N,普通交易,N,110,111,...,113Q1,692106.0,-316181.0,-102248,N,23.65,Y,167.13,N,無
184254,8429,金麗-KY,2024-06-07,上市,33.49,N,普通交易,N,110,111,...,113Q1,-160662.0,26748.0,8429,N,0.00,N,0.00,Y,有
184259,8488,吉源-KY,2024-06-07,上市,34.47,N,普通交易,N,110,111,...,113Q1,28963.0,712815.0,-215341,N,0.00,N,0.00,Y,有
184265,9933,中鼎,2024-06-07,上市,21.21,N,普通交易,N,110,111,...,113Q1,82309.0,2963017.0,3767612,N,10.26,Y,458.80,N,無


In [4]:
filtered_df['證券代碼'] = filtered_df['證券代碼'].apply(str)
pool = filtered_df['證券代碼'].unique()

C:\Users\to097\AppData\Local\Temp\ipykernel_31672\1657064431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['證券代碼'] = filtered_df['證券代碼'].apply(str)


In [5]:
data = tejapi.get('TWN/APRCD1',
                        coid = pool,
                        mdate = {'gte':'2020-01-03', 'lte':'2024-06-07'},
                        # 證券代碼、日期、收盤價(元)、報酬率%
                        opts = {'columns':['coid', 'mdate', 'close_adj']},
                        paginate = True,
                        )
data

,coid,mdate,close_adj
None,,,
0,1213,2020-01-03,6.10
1,1213,2020-01-06,5.99
2,1213,2020-01-07,6.16
3,1213,2020-01-08,6.16
4,1213,2020-01-09,5.93
...,...,...,...
151719,9955,2024-06-03,32.85
151720,9955,2024-06-04,32.95
151721,9955,2024-06-05,32.60


In [6]:
translation_dict = {
    "證券代碼": "coid",
    "簡稱": "company",
    "年月日": "mdate",
    "市場別": "Market",
    "產業類別": "Industry",
    "年月_每股淨值": "YearMonth_Net Value Per Share",
    "每股淨值": "Net Value Per Share",
    "符合指標數量": "Number of Indicators Met",
    "符合指標": "Indicators Met",
    "符合指標1 (Y/N)": "Indicator 1 Met (Y/N)",
    "指標1_交易方式": "Indicator 1_Transaction Method",
    "符合指標2 (Y/N)": "Indicator 2 Met (Y/N)",
    "指標2_最近稅後損益年月1": "Indicator 2_Recent Post-tax Profit/Loss Date 1",
    "指標2_最近稅後損益年月2": "Indicator 2_Recent Post-tax Profit/Loss Date 2",
    "指標2_最近稅後損益年月3": "Indicator 2_Recent Post-tax Profit/Loss Date 3",
    "指標2_最近三年稅後損益1": "Indicator 2_Recent Three Years Post-tax Profit/Loss 1",
    "指標2_最近三年稅後損益2": "Indicator 2_Recent Three Years Post-tax Profit/Loss 2",
    "指標2_最近三年稅後損益3": "Indicator 2_Recent Three Years Post-tax Profit/Loss 3",
    "符合指標3 (Y/N)": "Indicator 3 Met (Y/N)",
    "指標3_負債比率": "Indicator 3_Debt Ratio",
    "指標3_流動比率": "Indicator 3_Current Ratio",
    "符合指標4 (Y/N)": "Indicator 4 Met (Y/N)",
    "指標4_最近營業活動年月1": "Indicator 4_Recent Operating Activities Date 1",
    "指標4_最近營業活動年月2": "Indicator 4_Recent Operating Activities Date 2",
    "指標4_最近營業活動年月3": "Indicator 4_Recent Operating Activities Date 3",
    "指標4_最近期營業活動現金流量1": "Indicator 4_Recent Operating Cash Flow 1",
    "指標4_最近期營業活動現金流量2": "Indicator 4_Recent Operating Cash Flow 2",
    "指標4_最近期營業活動現金流量3": "Indicator 4_Recent Operating Cash Flow 3",
    "符合指標5 (Y/N)": "Indicator 5 Met (Y/N)",
    "指標5_資金貸與佔淨值比": "Indicator 5_Loans to Net Value Ratio",
    "符合指標6 (Y/N)": "Indicator 6 Met (Y/N)",
    "指標6_背書保證佔淨值比": "Indicator 6_Endorsement Guarantee to Net Value Ratio",
    "符合指標7 (Y/N)": "Indicator 7 Met (Y/N)",
    "指標7_其他": "Indicator 7_Others"
}

In [7]:
filtered_df.rename(columns=translation_dict, inplace=True)
filtered_df

C:\Users\to097\AppData\Local\Temp\ipykernel_31672\2898609827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns=translation_dict, inplace=True)


,coid,company,mdate,Market,Net Value Per Share,Indicator 1 Met (Y/N),Indicator 1_Transaction Method,Indicator 2 Met (Y/N),Indicator 2_Recent Post-tax Profit/Loss Date 1,Indicator 2_Recent Post-tax Profit/Loss Date 2,...,Indicator 4_Recent Operating Activities Date 3,Indicator 4_Recent Operating Cash Flow 1,Indicator 4_Recent Operating Cash Flow 2,Indicator 4_Recent Operating Cash Flow 3,Indicator 5 Met (Y/N),Indicator 5_Loans to Net Value Ratio,Indicator 6 Met (Y/N),Indicator 6_Endorsement Guarantee to Net Value Ratio,Indicator 7 Met (Y/N),Indicator 7_Others
0,1213,大飲,2022-01-03,上市,8.10,Y,變更交易,Y,107,108,...,110Q3,-67444.0,-3781.0,-36780,Y,30.73,N,0.00,Y,有
1,1235,興泰,2022-01-03,上市,33.15,N,普通交易,N,107,108,...,110Q3,41647.0,134987.0,63082,N,0.00,N,0.00,Y,有
3,1337,再生-KY,2022-01-03,上市,19.90,N,普通交易,N,107,108,...,110Q3,-340717.0,-215473.0,-266696,N,0.00,N,0.00,Y,有
4,1340,勝悅-KY,2022-01-03,上市,33.54,N,普通交易,N,107,108,...,110Q3,-194433.0,-312789.0,-232844,N,0.00,N,0.00,Y,有
5,1413,宏洲,2022-01-03,上市,8.34,N,普通交易,N,107,108,...,110Q3,5974.0,-39246.0,111106,N,0.00,N,0.00,N,無
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184252,8404,百和興業-KY,2024-06-07,上市,15.48,N,普通交易,N,110,111,...,113Q1,692106.0,-316181.0,-102248,N,23.65,Y,167.13,N,無
184254,8429,金麗-KY,2024-06-07,上市,33.49,N,普通交易,N,110,111,...,113Q1,-160662.0,26748.0,8429,N,0.00,N,0.00,Y,有
184259,8488,吉源-KY,2024-06-07,上市,34.47,N,普通交易,N,110,111,...,113Q1,28963.0,712815.0,-215341,N,0.00,N,0.00,Y,有
184265,9933,中鼎,2024-06-07,上市,21.21,N,普通交易,N,110,111,...,113Q1,82309.0,2963017.0,3767612,N,10.26,Y,458.80,N,無


In [8]:
filtered_df = filtered_df.merge(data[['mdate', 'coid', 'close_adj']], on=['mdate', 'coid'], how='left')
filtered_df

,coid,company,mdate,Market,Net Value Per Share,Indicator 1 Met (Y/N),Indicator 1_Transaction Method,Indicator 2 Met (Y/N),Indicator 2_Recent Post-tax Profit/Loss Date 1,Indicator 2_Recent Post-tax Profit/Loss Date 2,...,Indicator 4_Recent Operating Cash Flow 1,Indicator 4_Recent Operating Cash Flow 2,Indicator 4_Recent Operating Cash Flow 3,Indicator 5 Met (Y/N),Indicator 5_Loans to Net Value Ratio,Indicator 6 Met (Y/N),Indicator 6_Endorsement Guarantee to Net Value Ratio,Indicator 7 Met (Y/N),Indicator 7_Others,close_adj
0,1213,大飲,2022-01-03,上市,8.10,Y,變更交易,Y,107,108,...,-67444.0,-3781.0,-36780,Y,30.73,N,0.00,Y,有,8.420
1,1235,興泰,2022-01-03,上市,33.15,N,普通交易,N,107,108,...,41647.0,134987.0,63082,N,0.00,N,0.00,Y,有,29.699
2,1337,再生-KY,2022-01-03,上市,19.90,N,普通交易,N,107,108,...,-340717.0,-215473.0,-266696,N,0.00,N,0.00,Y,有,7.280
3,1340,勝悅-KY,2022-01-03,上市,33.54,N,普通交易,N,107,108,...,-194433.0,-312789.0,-232844,N,0.00,N,0.00,Y,有,11.550
4,1413,宏洲,2022-01-03,上市,8.34,N,普通交易,N,107,108,...,5974.0,-39246.0,111106,N,0.00,N,0.00,N,無,10.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53796,8404,百和興業-KY,2024-06-07,上市,15.48,N,普通交易,N,110,111,...,692106.0,-316181.0,-102248,N,23.65,Y,167.13,N,無,21.550
53797,8429,金麗-KY,2024-06-07,上市,33.49,N,普通交易,N,110,111,...,-160662.0,26748.0,8429,N,0.00,N,0.00,Y,有,10.100
53798,8488,吉源-KY,2024-06-07,上市,34.47,N,普通交易,N,110,111,...,28963.0,712815.0,-215341,N,0.00,N,0.00,Y,有,16.200
53799,9933,中鼎,2024-06-07,上市,21.21,N,普通交易,N,110,111,...,82309.0,2963017.0,3767612,N,10.26,Y,458.80,N,無,48.800


In [9]:
filtered_df= filtered_df.dropna(subset=['close_adj'])
filtered_df = filtered_df.replace({'Y': 1, 'N': 0,})
filtered_df.fillna(0, inplace=True)
filtered_df

,coid,company,mdate,Market,Net Value Per Share,Indicator 1 Met (Y/N),Indicator 1_Transaction Method,Indicator 2 Met (Y/N),Indicator 2_Recent Post-tax Profit/Loss Date 1,Indicator 2_Recent Post-tax Profit/Loss Date 2,...,Indicator 4_Recent Operating Cash Flow 1,Indicator 4_Recent Operating Cash Flow 2,Indicator 4_Recent Operating Cash Flow 3,Indicator 5 Met (Y/N),Indicator 5_Loans to Net Value Ratio,Indicator 6 Met (Y/N),Indicator 6_Endorsement Guarantee to Net Value Ratio,Indicator 7 Met (Y/N),Indicator 7_Others,close_adj
0,1213,大飲,2022-01-03,上市,8.10,1,變更交易,1,107,108,...,-67444.0,-3781.0,-36780,1,30.73,0,0.00,1,有,8.420
1,1235,興泰,2022-01-03,上市,33.15,0,普通交易,0,107,108,...,41647.0,134987.0,63082,0,0.00,0,0.00,1,有,29.699
2,1337,再生-KY,2022-01-03,上市,19.90,0,普通交易,0,107,108,...,-340717.0,-215473.0,-266696,0,0.00,0,0.00,1,有,7.280
3,1340,勝悅-KY,2022-01-03,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.00,0,0.00,1,有,11.550
4,1413,宏洲,2022-01-03,上市,8.34,0,普通交易,0,107,108,...,5974.0,-39246.0,111106,0,0.00,0,0.00,0,無,10.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53796,8404,百和興業-KY,2024-06-07,上市,15.48,0,普通交易,0,110,111,...,692106.0,-316181.0,-102248,0,23.65,1,167.13,0,無,21.550
53797,8429,金麗-KY,2024-06-07,上市,33.49,0,普通交易,0,110,111,...,-160662.0,26748.0,8429,0,0.00,0,0.00,1,有,10.100
53798,8488,吉源-KY,2024-06-07,上市,34.47,0,普通交易,0,110,111,...,28963.0,712815.0,-215341,0,0.00,0,0.00,1,有,16.200
53799,9933,中鼎,2024-06-07,上市,21.21,0,普通交易,0,110,111,...,82309.0,2963017.0,3767612,0,10.26,1,458.80,0,無,48.800


In [10]:
X = sm.add_constant(filtered_df[['Net Value Per Share', 
                                 'Indicator 2_Recent Three Years Post-tax Profit/Loss 1' ,
                                 'Indicator 2_Recent Three Years Post-tax Profit/Loss 2' ,
                                 'Indicator 2_Recent Three Years Post-tax Profit/Loss 3', 
                                 'Indicator 3_Debt Ratio','Indicator 3_Current Ratio' ,
                                 'Indicator 4_Recent Operating Cash Flow 1',
                                 'Indicator 4_Recent Operating Cash Flow 2',
                                 'Indicator 4_Recent Operating Cash Flow 3',
                                 'Indicator 5_Loans to Net Value Ratio',
                                 'Indicator 6_Endorsement Guarantee to Net Value Ratio',
                                 'Indicator 7 Met (Y/N)']])
Y = filtered_df[['close_adj']]

In [16]:
df = filtered_df[filtered_df['coid'] =='1340']
df

,coid,company,mdate,Market,Net Value Per Share,Indicator 1 Met (Y/N),Indicator 1_Transaction Method,Indicator 2 Met (Y/N),Indicator 2_Recent Post-tax Profit/Loss Date 1,Indicator 2_Recent Post-tax Profit/Loss Date 2,...,Indicator 4_Recent Operating Cash Flow 1,Indicator 4_Recent Operating Cash Flow 2,Indicator 4_Recent Operating Cash Flow 3,Indicator 5 Met (Y/N),Indicator 5_Loans to Net Value Ratio,Indicator 6 Met (Y/N),Indicator 6_Endorsement Guarantee to Net Value Ratio,Indicator 7 Met (Y/N),Indicator 7_Others,close_adj
3,1340,勝悅-KY,2022-01-03,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.0,0,0.0,1,有,11.55
84,1340,勝悅-KY,2022-01-04,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.0,0,0.0,1,有,11.55
165,1340,勝悅-KY,2022-01-05,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.0,0,0.0,1,有,11.55
246,1340,勝悅-KY,2022-01-06,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.0,0,0.0,1,有,11.50
327,1340,勝悅-KY,2022-01-07,上市,33.54,0,普通交易,0,107,108,...,-194433.0,-312789.0,-232844,0,0.0,0,0.0,1,有,11.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53434,1340,勝悅-KY,2024-06-03,上市,24.26,0,普通交易,0,110,111,...,-362577.0,-272617.0,-82055,0,1.8,0,0.0,1,有,10.20
53508,1340,勝悅-KY,2024-06-04,上市,24.26,0,普通交易,0,110,111,...,-362577.0,-272617.0,-82055,0,1.8,0,0.0,1,有,10.10
53582,1340,勝悅-KY,2024-06-05,上市,24.26,0,普通交易,0,110,111,...,-362577.0,-272617.0,-82055,0,1.8,0,0.0,1,有,10.00
53656,1340,勝悅-KY,2024-06-06,上市,24.26,0,普通交易,0,110,111,...,-362577.0,-272617.0,-82055,0,1.8,0,0.0,1,有,10.05


In [11]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              close_adj   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     7890.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:57:21   Log-Likelihood:            -2.1303e+05
No. Observations:               42328   AIC:                         4.261e+05
Df Residuals:                   42315   BIC:                         4.262e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------